In [ ]:
import datetime
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import selenium.webdriver.common.by as by
import os
import csv
import uuid as guid

class csvItem: 
    id:guid
    seller:str=""
    name:str = ""
    priceWithCard:str
    price:str
    priceOriginal:str
    description:str= ""
    rating:float =0.0
    otziv:str= ""
    link:str= ""

csv_file = "name,rating,otziv,textLable,link,description,other"
csvPath= 'C://Users//User//Desktop//py11//py//csv//'
directoryPath= 'C://Users//User//Desktop//py11//py//ozon_pages//'
directoryData= 'C://Users//User//Desktop//py11//py//inner_data//'

d_format =datetime.datetime.strftime(datetime.datetime.utcnow(),'%m-%d-%Y')

def downloadFiles():
    dr = webdriver.Chrome(ChromeDriverManager().install())
    domen_url = "https://www.ozon.ru/api/composer-api.bx/page/json/v2?page="
    page_kopter = "&url=/category/kvadrokoptery-7160/?type=54331"
    finish_message =""
    i= 0
    totalScorePage = 0
    totalScoreProduct = 0
    #TODO: сделать отлов по сообщению о том что больше записей нету
    while i<=32:
        url=domen_url+str(i)+page_kopter
        dr.get(url)
        content = dr.page_source
        content = dr.find_element(by=by.By.TAG_NAME, value='pre').text
        path = directoryPath + d_format +'//' +'outputfile'+str(i)+'.json'
        with open(path, 'w') as outf:
            json.dump(content, outf)
        
        #Here we get only data about products
        #and write in new json
        with open(path, 'r') as inf:
            data = json.load(inf)
        
        jj = json.loads(data)
        J_widgetStates = jj['widgetStates']

        pathData = directoryData + d_format +'//'+'innerfile'+str(i)+'.json'
        with open(pathData, 'w') as wf:
            json.dump(J_widgetStates, wf)
        #end

        i+=1
        print("download "+ str(i) + "page")
    else:
        print("download pages finish")

listsymbl = ['']
isFilesExist = os.listdir(directoryPath+ '11-18-2022') 

if isFilesExist == []:
    downloadFiles()

innerFiles = os.listdir(directoryData + '11-18-2022' +'//')
with open(csvPath + 'csv_items_'+'11-18-2022'+'.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['id','name', 'seller', 'priceOriginal', 'priceWithCard', 'price', 'rating', 'otziv', 'link', 'description'])
    #Из за большой вложенности, приходится его каждый раз в json конвертитьвать
    #каждый раз проваливаясь на уровень ниже
    for i in innerFiles:
        print(i)
        inner_data_f = open(directoryData + '11-18-2022' +'//' + i,'r')
        inner_data_json = json.load(inner_data_f)
        searchResultsV2 = inner_data_json['searchResultsV2-252189-default-1']
        searchResultsV2_json = json.loads(searchResultsV2)
        resultsV2_items = searchResultsV2_json['items']
        for v2_item in resultsV2_items: 
            ite = csvItem 
            ite.id = guid.uuid4()
            ite.link = v2_item['action']['link']
            ite.seller = v2_item['multiButton']['ozonSubtitle']['textAtomWithIcon']['text']
            prices = v2_item['rightState']
            priceWithTitle = [item for item in v2_item['rightState'] if item['atom']['type']== 'priceWithTitle']
            if(len(priceWithTitle) != 0):
                ite.priceWithCard = priceWithTitle[0]['atom']['priceWithTitle']['price']
            prices = [item for item in v2_item['rightState'] if item['atom']['type']== 'price']
            if(len(prices) !=0):
                if('originalPrice' in prices[0]['atom']['price'].keys()):
                    ite.priceOriginal = prices[0]['atom']['price']['originalPrice']
                if('price' in prices[0]['atom']['price'].keys()):
                    ite.price = prices[0]['atom']['price']['price']

            main_items = v2_item['mainState']

            for main_item in main_items:
                if ('id' in main_item.keys()):
                    if main_item['id'] == "name":
                        ite.name = main_item['atom']['textAtom']['text']
                    if main_item['id'] == "topAttributes":
                        ite.description = main_item['atom']['textAtom']['text']                     
                if('atom' in main_item.keys() and (len(main_item)==1 or main_item != [])):
                    if(main_item['atom']['type'] == 'labelList'):
                        if((main_item['atom']['labelList']['items'][0]['title'] != 'Новинка')
                         and (main_item['atom']['labelList']['items'][0]['title'] != 'Бестселлер')
                         and ("бонусов" not in main_item['atom']['labelList']['items'][0]['title'])):
                            if(len(main_item['atom']['labelList']['items']) == 2):
                                ite.rating = main_item['atom']['labelList']['items'][0]['title']
                                ite.otziv = main_item['atom']['labelList']['items'][1]['title']
                            if(len(main_item['atom']['labelList']['items']) == 1):
                                try:
                                    if(int(main_item['atom']['labelList']['items'][0]['title'])<=5.0):
                                        ite.rating = main_item['atom']['labelList']['items'][0]['title']
                                    else:
                                        ite.otziv = main_item['atom']['labelList']['items'][1]['title']
                                except:
                                    continue
                
            spamwriter.writerow([ite.id, ite.name, ite.seller,  ite.priceOriginal, ite.priceWithCard, ite.price, ite.rating, ite.otziv, ite.link, ite.description])
         
    csvfile.close()

In [17]:
import pandas as pd
csvPath= 'C://Users//User//Desktop//py11//py//csv//csv_items_11-18-2022.csv'
df = pd.read_csv(csvPath)    
for s in ["<font color='#001a34'>", "<font color='#005bff'>", "</font>", "<b>", "</b>", "продавец", "Из-за рубежа", "доставка"]:
    df['seller'] = df['seller'].str.replace(s, '')

for i, row in df.iterrows():
    df.at[i,'seller'] = df.at[i,'seller'][df.at[i,'seller'].index(",")+1:]

for d in ["<font color='#001a34'>","</font>", "<br>", "</br>"]:
    df['description'] = df['description'].str.replace(d, '')
        
df.to_csv('csv_items_11-18-2022_clear.csv',sep=',')

In [ ]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import selenium.webdriver.common.by as by
import json
import datetime
import time


def downloadCharacters(p):
    df = pd.read_csv(p)
    p=1009
    for j in df['link'].values[1009:]:
        #indexes = [i for i, symb in enumerate(j) if symb=='/']
        #substr = j[0:int(indexes[2])]
        url = domen_url+str(j)+'&layout_container=pdpPage2column&layout_page_index=2&sh=PAXdSbLYGQ'
        time.sleep(7)
        dr.get(url)
        content = dr.page_source
        content = dr.find_element(by=by.By.TAG_NAME, value='pre').text
        path = characterPath + d_format +'//' +'character_file'+str(p)+'.json'
        with open(path, 'w') as charf:
            json.dump(content, charf)
        p+=1
        print('download by endpont'+str(j))

csvPath= 'C://Users//User//Desktop//py11//py//csv//csv_items_'
d_format =datetime.datetime.strftime(datetime.datetime.utcnow(),'%m-%d-%Y')
domen_url = "https://www.ozon.ru/api/composer-api.bx/page/json/v2?url="
dr = webdriver.Chrome(ChromeDriverManager().install())
characterPath= 'C://Users//User//Desktop//py11//py//character_kopters//'

downloadCharacters(csvPath+d_format+".csv")       

In [ ]:

class Action:
    def __init__(self, behavior:str, link:str):
        self.behavior=behavior
        self.link=link;

class TestInfo:
    def __init__(self, automatizationId:str):
        self.automatizationId=automatizationId;

class TextAtom:
    def __init__(self, text:str, textStyle:str, 
    textColor:str, maxLines:int, testInfo:TestInfo):
        self.text = text
        self.textStyle = textStyle
        self.textColor = textColor
        self.maxLines = maxLines
        self.testInfo = testInfo;

class Raiting:
    def __init__(self, raiting:float, count:str, theme:str,
    action:Action):
        self.raiting=raiting
        self.count=count
        self.theme=theme
        self.action=action;

class Price:
    def __init__(self, price:str, priceColor:str,
    originalPrice:str, originalPriceColor:str, theme:str, strikethroughColor:str):
        self.price=price
        self.priceColor=priceColor
        self.originalPrice=originalPrice
        self.originalPriceColor = originalPriceColor
        self.theme =theme
        self.strikethroughColor=strikethroughColor;

class Atom:
    def __init__(self, type:str, textAtom:TextAtom, raiting:Raiting, price:Price):
        self.type = type
        self.textAtom = textAtom
        self.raiting=raiting
        self.price=price;

class MainState:
    def __init__(self, type:str, id:str, atom:Atom):
        self.type = type
        self.id = id
        self.atom = atom;

class Item:
    def __init__(self, action:Action, backgroundColor:str, 
    isAdult:bool, mainState:MainState = []):
        self.action = action
        self.backgroundColor = backgroundColor
        self.isAdult = isAdult
        self.mainState = mainState;

    def __init__(self, j):
         self.__dict__ = json.loads(j)
